In [ ]:
import datetime

import holidays
from sklearn.cluster import HDBSCAN, KMeans
import pandas as pd

co_hol = holidays.Colombia()

Winter = [5, 6, 7, 8, 9, 10, 11]
Summer = [1, 2, 3, 4, 12]

In [ ]:
df = pd.read_excel('data/Generacion_(kWh)_2024.xlsx', engine='openpyxl', parse_dates=["Fecha"])
df_ = df.set_index([
    col for col in df.columns if col not in [str(h) for h in range(24)]
])
df_.columns = range(24)
df_ = df_.stack().reset_index()
df_.rename(columns={'level_10': "hours", 0: "generacion"}, inplace=True)
df_["datetime"] = pd.to_datetime(df_["Fecha"]) + pd.to_timedelta(df_["hours"], unit='h')
df_ = df.set_index([
    col for col in df.columns if col not in [str(h) for h in range(24)]
])
df_.columns = range(24)
df_ = df_.stack().reset_index()
df_.rename(columns={'level_10': "hours", 0: "generacion"}, inplace=True)
df_["datetime"] = pd.to_datetime(df_["Fecha"]) + pd.to_timedelta(df_["hours"], unit='h')
df_

In [ ]:

df_agg = df_.groupby(["Fecha", "Tipo Generación"]).agg({
    "generacion": "sum",
}).reset_index()
# }).reset_index()
df_agg["generacion"] = df_agg["generacion"]*1E-6 
df_agg = df_agg.pivot(index="Fecha", columns="Tipo Generación", values="generacion")
df_agg.columns.name=None
df_agg["TOTAL_GEN"] = df_agg.sum(axis=1)
# Generación in GWh
df_agg = df_agg.reset_index()

df_agg["day_of_week"] = df_agg["Fecha"].dt.dayofweek
# df_agg["week_of_year"] = df_agg["Fecha"].dt.isocalendar().week
df_agg["winter"] = df_agg["Fecha"].dt.month.apply(lambda x:  1 if x in Winter else 0)
# df_agg["is_holiday"] = df_agg["Fecha"].apply(lambda x: 1 if co_hol.get(x) else 0)
df_agg.fillna(0, inplace=True)

In [ ]:
excep_columns = ["Fecha", "Tipo Generación", "TOTAL_GEN"]
excep_columns = excep_columns + ["COGENERADOR", "EOLICA", "SOLAR"]

df_agg[[col for col in df_agg.columns if col not in excep_columns]]

In [ ]:
# Fit K-means for different values of k
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import cdist
distortions = []
inertias = []
mapping1 = {}
mapping2 = {}

K = range(1, 20)
X = df_agg[[col for col in df_agg.columns if col not in excep_columns]].values
X = X.astype(float)
for k in K:
    kmeanModel = KMeans(n_clusters=k, random_state=42).fit(X)
    
    # Calculate distortion as the average squared distance from points to their cluster centers
    distortions.append(sum(np.min(cdist(X, kmeanModel.cluster_centers_, 'euclidean'), axis=1)**2) / X.shape[0])
    
    # Inertia is calculated directly by KMeans
    inertias.append(kmeanModel.inertia_)
    
    # Store the mappings for easy access
    mapping1[k] = distortions[-1]
    mapping2[k] = inertias[-1]

print("Distortion values:")
for key, val in mapping1.items():
    print(f'{key} : {val}')
# Plotting the graph of k versus Distortion
plt.plot(K, distortions, 'bx-')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Distortion')
plt.title('The Elbow Method using Distortion')
plt.grid()
plt.show()

In [ ]:
# clusterer = HDBSCAN()
clusterer = KMeans(n_clusters=7)
clusterer.fit(X=df_agg[[col for col in df_agg.columns if col not in excep_columns]])

In [ ]:
df_agg["cluster"] = clusterer.labels_.astype(str)
df_agg["day_name"] = df_agg["Fecha"].dt.day_name()

In [ ]:
df_agg.cluster.nunique()

In [ ]:
# with open("")

df_agg.groupby("cluster").agg({"Fecha": lambda x : list(x)}).to_dict()

In [ ]:
df_agg.groupby("cluster").agg({"Fecha": "count"}).to_dict()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
fig = px.bar(
    data_frame=df_agg,
    x="Fecha",
    y="TOTAL_GEN",
    color="cluster",
    color_discrete_sequence=px.colors.qualitative.Set1,
    # hover_data=["week_of_year", "day_of_week", "is_holiday", "day_name"],
)
fig.add_traces(
    [
        go.Scatter(
            x=df_agg["Fecha"],
            y=df_agg["TERMICA"],
            mode='lines',
            line=dict(color='black', width=1),
            name="Termica",
        ),
        go.Scatter(
            x=df_agg["Fecha"],
            y=df_agg["HIDRAULICA"],
            mode='lines',
            line=dict(color='#ed6f1c', width=1),
            name="hidro",
        ),
    ]
)


fig.show()

In [ ]:
centers = pd.DataFrame(
    data=clusterer.cluster_centers_,
    columns=[col for col in df_agg.columns if col not in excep_columns + ["cluster", "day_name"]],
    index=range(clusterer.n_clusters)
)
centers
df_agg["HIDRO_ERROR"] = df_agg.apply(
    lambda x: (x["HIDRAULICA"] - centers.loc[int(x["cluster"]), "HIDRAULICA"])**2, axis=1
)
df_agg["TERMICA_ERROR"] = df_agg.apply(
    lambda x: (x["TERMICA"] - centers.loc[int(x["cluster"]), "TERMICA"])**2, axis=1
)
df_agg["TOTAL_ERROR"] = df_agg["HIDRO_ERROR"] + df_agg["TERMICA_ERROR"]
min_error_dates = df_agg.loc[df_agg.groupby("cluster")["TOTAL_ERROR"].idxmin(), ["cluster", "Fecha", "TOTAL_ERROR"]]
min_error_dates